<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->

# <font color='blue'>Data Science Academy</font>

# <font color='blue'>Processamento de Linguagem Natural com Transformers</font>

## <font color='blue'>Projeto 6</font>

## <font color='blue'>Sistema de Busca em Documentos com LLM e VectorDB</font>

## Instalando e Carregando Pacotes

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.11.6


In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark.
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
%pip install -q -U watermark


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install -q -U langchain openai

In [ ]:
!pip install -q pinecone-client

In [ ]:
!pip install -q unstructured

In [ ]:
!pip install -q unstructured_inference

In [ ]:
!pip install -q pypdf

In [ ]:
!pip install -q PyPDF2

In [ ]:
!pip install -q pdf2image

In [ ]:
!pip install -q pikepdf

In [ ]:
!pip install -q tiktoken

In [ ]:
# Ambiente Linux (como o Google Colab) requer o pacote abaixo
# !apt-get install poppler-utils

In [ ]:
# Imports
import os
import openai
import langchain
import pinecone
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone

/Users/dmpm/anaconda3/lib/python3.11/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

pinecone : 2.2.4
langchain: 0.0.353
openai   : 1.6.1



## Extraindo Texto de Documentos

In [ ]:
# Pasta de dados (arquivos PDF)
pasta_dados = r"dados"

In [ ]:
# Função
def carrega_arquivos(directory):
    
    loader = DirectoryLoader(directory)
    
    documents = loader.load()

    return documents

In [ ]:
# Executa a função
dsa_documentos = carrega_arquivos(pasta_dados)

In [ ]:
print(len(dsa_documentos))

2


In [ ]:
print(dsa_documentos)

[Document(page_content='Inteligência Artificial Dispara Como Tecnologia Mais Relevante Para 2024\n\nUma pesquisa feita com líderes globais de tecnologia dos EUA, Brasil, Reino Unido, China e Ìndia pelo IEEE (Instituto dos Engenheiros Eletrônicos e Eletricistas), organização mundial técnico- profissional dedicada ao avanço da tecnologia em benefício da humanidade, mostra que para 65% dos entrevistados, a IA generativa vai continuar a dominar o cenário tecnológico a partir de 2024.\n\nO estudo pediu ainda para 350 CIOs e executivos de TI definirem três áreas principais na tecnologia no próximo ano e as respostas foram:\n\n(65%) Inteligência Artificial (IA), incluindo IA preditiva e generativa, machine learning (ML) e processamento de linguagem natural (PLN);\n\n(28%) Realidade estendida (XR), incluindo metaverso, realidade aumentada (AR), realidade virtual (VR) e realidade mista (MR); e\n\n(24%) computação em nuvem\n\nOutras tecnologias importantes citadas para 2024 foram 5G (22%) e veíc

## Pré-Processamento dos Dados de Texto

In [ ]:
# Função
def split_docs(documents, chunk_size = 1000, chunk_overlap = 20):

    text_splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size,
                                                   chunk_overlap = chunk_overlap)

    docs = text_splitter.split_documents(documents)

    return docs

In [ ]:
dsa_docs = split_docs(dsa_documentos)

In [ ]:
print(len(dsa_docs))

14


In [ ]:
print(dsa_docs[1].page_content)

Os participantes da pesquisa também foram perguntados qual porcentagem de empregos em toda economia global em 2024 será aumentada por software baseado IA e a maioria foi otimista. De 26% a 50% dos empregos foram citados por 41% dos respondentes. Mais de um quarto (28%) citou 1-25% dos empregos, outros 26% citaram de 51-75% dos empregos, e 5% citaram 76-100% dos empregos.

Telecomunicações (20%), Manufatura (18%), Energia (17%), Entretenimento e Serviços Bancários e Financeiros (ambos com 16%) serão os setores mais impactados pela tecnologia em 2024.

O levantamento também mostra que as áreas como Tecnologia Nuclear (28%), 5G (22%), Robótica (21%), Realidade Estendida (20%) e Internet Industrial das Coisas (IoT) (19%) terão atenção das companhias. O levantamento apurou ainda que para 44% dos brasileiros entrevistados, 6G deve se tornar padrão nos próximos 3 a 5 anos.


## Gerando as Embeddings

Crie sua API na OpenAI e coloque no arquivo .env na mesma pasta onde está este Jupyter Notebook.

https://platform.openai.com/

https://platform.openai.com/api-keys

https://platform.openai.com/docs/quickstart?context=python

In [ ]:
!pip install -q python-dotenv

In [ ]:
# Carregando as variáveis de ambiente definidas no arquivo .env
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
embeddings = OpenAIEmbeddings(api_key = os.environ['OPENAI_API_KEY'])

In [ ]:
query_result = embeddings.embed_query("Aprendendo sobre LLMs com a DSA")

In [ ]:
len(query_result)

1536

In [ ]:
query_result

[-0.005041217572552165,
 0.027037697669790205,
 -0.0038109818608999153,
 -0.04395859439051072,
 -0.025841825433796817,
 0.021938060093142125,
 -0.01503774286435609,
 0.007498252881121428,
 -0.01708584512917345,
 -0.013237062280895537,
 0.0156562966020868,
 0.001833326350997773,
 -0.0045326288195750115,
 -0.0026443198597851346,
 -0.0020085834529023927,
 0.009381407203147151,
 0.019298894871121144,
 -0.005835028994264121,
 0.033511900349825555,
 -0.007312686852934475,
 -0.0036700890495614323,
 -0.01462537370586895,
 0.012707854976861766,
 -0.023367608620228773,
 0.000304122551243345,
 -0.012591016831315137,
 0.0075051260419145005,
 -0.015601315041032623,
 -0.013848743789155774,
 0.005113382501250325,
 0.04420601700319012,
 -0.008178662737683291,
 -0.01228173949678848,
 -0.014694101588509274,
 -0.003999984469483404,
 -0.027862435986764487,
 0.018130515278300046,
 -0.007388287896367871,
 0.02141572594990143,
 -0.012082428078337983,
 0.001075597277219572,
 0.0028848686131795168,
 0.00716148

## Inicializando o Banco de Dados Vetorial

Cria sua conta gratuita no Pinecone, valide a conta, crie um índice chamado dsa-index e crie a chave da API. A chave da API deve ser colocada no arquivo .env na mesma pasta onde está este Jupyter Notebook.

https://www.pinecone.io/

https://app.pinecone.io/

In [ ]:
# Inicializando o Pinecone
pinecone.init(api_key = os.environ['PINECONE_API_KEY'], environment = 'gcp-starter')

In [ ]:
# Nome do índice
index_name = "dsa-index"

In [ ]:
# Convertendo os dados de texto em vetores e armazenando no índice
index = Pinecone.from_documents(dsa_docs, embeddings, index_name = index_name)

## Construindo o Sistema de Busca Semântica em Documentos

In [ ]:
# Função
def busca_docs_similares(query, k = 1, score = True):

    if score:
        similar_docs = index.similarity_search_with_score(query, k = k)
    else:
        similar_docs = index.similarity_search(query, k = k)

    return similar_docs

## Testando o Sistema

In [ ]:
query1 = "Processamento de Linguagem Natural no Centro das Atenções"

In [ ]:
similar_docs_1 = busca_docs_similares(query1)

In [ ]:
similar_docs_1

[(Document(page_content='Processamento de Linguagem Natural no Centro das Atenções\n\nUm sinal claro do impacto do ChatGPT está no interesse que os usuários da O’Reilly mostraram em PLN, que teve o maior crescimento ano a ano em 42%, e em aprendizado profundo (Deep Learning), que seguiu em 23%. O PLN tem muitos usos, desde chatbots automatizados até ferramentas de geração e escrita de código, portanto, o interesse não é surpreendente.\n\nOs profissionais também têm procurado cada vez mais conteúdo relacionado a Transformers, o modelo de IA que ajudou a alimentar o PLN, refletindo ainda mais o impacto do ChatGPT. O interesse em PLN e aprendizado profundo só aumentará à medida que o potencial dessas ofertas crescer e novos projetos aparecerem.', metadata={'source': 'dados/ArtigoDSA1.pdf'}),
  0.886582792)]

In [ ]:
query2 = "Entretenimento e Serviços Bancários e Financeiros"

In [ ]:
similar_docs_2 = busca_docs_similares(query2)

In [ ]:
similar_docs_2

[(Document(page_content='Os participantes da pesquisa também foram perguntados qual porcentagem de empregos em toda economia global em 2024 será aumentada por software baseado IA e a maioria foi otimista. De 26% a 50% dos empregos foram citados por 41% dos respondentes. Mais de um quarto (28%) citou 1-25% dos empregos, outros 26% citaram de 51-75% dos empregos, e 5% citaram 76-100% dos empregos.\n\nTelecomunicações (20%), Manufatura (18%), Energia (17%), Entretenimento e Serviços Bancários e Financeiros (ambos com 16%) serão os setores mais impactados pela tecnologia em 2024.\n\nO levantamento também mostra que as áreas como Tecnologia Nuclear (28%), 5G (22%), Robótica (21%), Realidade Estendida (20%) e Internet Industrial das Coisas (IoT) (19%) terão atenção das companhias. O levantamento apurou ainda que para 44% dos brasileiros entrevistados, 6G deve se tornar padrão nos próximos 3 a 5 anos.', metadata={'source': 'dados/ArtigoDSA2.pdf'}),
  0.804885566)]

# Fim